In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import numpy as np
from matplotlib import pyplot as plt
import pickle
import os
import glob
import io
from PIL import Image
from torchvision import transforms
import sys
from multiprocessing import Pool
import itertools
from matplotlib import gridspec

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
import sg_utils

Using device cuda:0


In [4]:
ROOT_ACTIVATIONS_DIR = "/int_comp_out/activations"
IMAGENET_DIR = "/scratch_local/datasets/ImageNet2012"
SEED = 0
N_BATCHES = 20

# Utils

In [5]:
resize_and_crop = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
    ]
)

def open_image(path):
    return np.array(resize_and_crop(Image.open(os.path.join(IMAGENET_DIR, path))))

def zip_chunked(*seqs, size):
    def get_chunks(seq, size):
        return list(seq[pos:pos + size] for pos in range(0, len(seq), size))
    return itertools.chain(*itertools.chain(*zip(*[get_chunks(seq, size) for seq in seqs])))

In [6]:
def get_task_image(min_reference_paths, max_reference_paths, query_paths):
    fig, axes = plt.subplot_mosaic("""
    ABCJLMN
    DEF OPQ
    GHIKRST
    """)
    image_size = 1
    fig.set_size_inches(7 * image_size, 3 * image_size)
    for k in axes:
        axes[k].axis("off")

    assert len(min_reference_paths) == 9
    assert len(max_reference_paths) == 9
    assert len(query_paths) == 2

    image_paths = min_reference_paths + max_reference_paths + query_paths
    images = prepare_pool.pool.map(open_image, image_paths)
    min_reference_images = images[:len(min_reference_paths)]
    max_reference_images = images[len(min_reference_paths):len(min_reference_paths) + len(max_reference_paths)]
    query_images = images[-len(query_paths):]

    for mri, ak in zip(min_reference_images, "ABCDEFGHI"):
        axes[ak].imshow(mri)
    for mri, ak in zip(max_reference_images, "LMNOPQRST"):
        axes[ak].imshow(mri)
    for qi, ak in zip(query_images, "JK"):
        axes[ak].imshow(qi)
    plt.tight_layout()
    buffer = io.BytesIO()
    plt.savefig(buffer, format="png", bbox_inches="tight")
    buffer.seek(0)
    plt.close()

    return buffer.read()


def mute_process():
    sys.stderr = open(os.devnull, 'w')


def prepare_pool():
    if not hasattr(prepare_pool, "pool"):
        prepare_pool.pool = None
    if prepare_pool.pool is None:
        prepare_pool.pool = Pool(processes=8, initializer=mute_process)
    else:
        prepare_pool.pool.close()
        prepare_pool.pool = Pool(processes=8, initializer=mute_process)


def get_task_paths(activations_df, unit, percentile, n_batches=N_BATCHES):
    np.random.seed(SEED)

    num_images_total = len(activations_df)
    query_start_idx_min = 9 * n_batches + int((100 - percentile) / 100 * num_images_total)
    query_stop_idx_max = num_images_total - query_start_idx_min
    query_stop_idx_min, query_start_idx_max = query_start_idx_min + n_batches, query_stop_idx_max - n_batches

    (min_queries, max_queries), (min_refs, max_refs) = sg_utils.extract_stimuli_range(
        activations_df,
        unit,
        [
            (query_start_idx_min, query_stop_idx_min, query_start_idx_max, query_stop_idx_max),
            (
                0,
                n_batches * 9,
                num_images_total - (n_batches * 9),
                num_images_total,
            ),
        ],
    )

    # Combine the lists - both lists go from least to most, so min list starts with
    # queries (the first / last ten images land in the batch from which queries are
    # sourced, so this is fine).
    min_lists = sg_utils.make_fair_batches(min_queries + min_refs, n_batches, reverse=True)
    max_lists = sg_utils.make_fair_batches(max_refs + max_queries, n_batches)

    return [(mil[:9], mal[:9], [mil[9], mal[9]]) for mil, mal in zip(min_lists, max_lists)]


def get_exemplar_paths(activations_df, unit, topk):
    activations_df = activations_df[["path", str(unit)]]
    activations_df = activations_df.sort_values(str(unit), ascending=True)

    return list(activations_df.iloc[:topk]["path"].array), list(activations_df.iloc[-topk:]["path"].array)


def get_exemplars_image(min_reference_paths, max_reference_paths):
    assert len(min_reference_paths) == len(max_reference_paths)
    n_rows = int(np.ceil(len(max_reference_paths) / 4))
    image_size = 1

    fig = plt.figure()
    outer = gridspec.GridSpec(1, 2, width_ratios = [1, 1], figure=fig)
    gs1 = gridspec.GridSpecFromSubplotSpec(n_rows, 4, subplot_spec = outer[0], hspace=0)
    gs2 = gridspec.GridSpecFromSubplotSpec(n_rows, 4, subplot_spec = outer[1], hspace=0)
    fig.set_size_inches(8 * image_size, n_rows * image_size)

    image_paths = min_reference_paths + max_reference_paths
    images = prepare_pool.pool.map(open_image, image_paths)
    min_images = images[:len(min_reference_paths)]
    max_images = images[len(min_reference_paths):]

    for gs, images in zip((gs1, gs2), (min_images, max_images)):
        for cell, im in zip(gs, images):
            ax = plt.subplot(cell)
            ax.imshow(im)
            ax.axis("off")

    plt.tight_layout()
    buffer = io.BytesIO()
    plt.savefig(buffer, format="png", bbox_inches="tight")
    buffer.seek(0)
    plt.close()

    return buffer.read(), (n_rows, 8)

In [7]:
def setup_task_ui(model_name):
    activations_dir = os.path.join(ROOT_ACTIVATIONS_DIR, model_name)
    available_layers = list(glob.glob(os.path.join(activations_dir, "*.pkl"))) + list(glob.glob(os.path.join(activations_dir, "*.csv")))
    available_layers = sorted(list(set([os.path.basename(al)[:-4] for al in available_layers])))
    if len(available_layers) == 0:
        raise ValueError(f"Could not find activations for model {model_name}")

    layer_widget = widgets.Dropdown(options=available_layers, description="Layer:")
    unit_widget = widgets.Dropdown(options=[], description="Unit:")
    percentile_widget = widgets.IntSlider(min=51, max=100, step=1, value=100, description="Query Perc.:", continuous_update=False)
    task_widget = widgets.Dropdown(options=list(range(N_BATCHES)), description="Task:")
    image_widget = widgets.Image(value=memoryview(b""), width=800, height=800 / 7 * 3)

    current_layer_activations_df = None
    current_layer = None
    current_unit = None
    current_percentile = None
    current_task = None
    current_tasks = None

    def show_examples(task):
        if current_tasks:
            layer_widget.disabled = unit_widget.disabled = percentile_widget.disabled = True
            data = get_task_image(*current_tasks[task])
            image_widget.value = memoryview(data)
            display(image_widget)
            layer_widget.disabled = unit_widget.disabled = percentile_widget.disabled = False

    def update_tasks(unit, percentile):
        nonlocal current_tasks
        current_tasks = get_task_paths(current_layer_activations_df, unit, percentile)

    def update_layer_choice(layer, unit, percentile, task):
        nonlocal current_layer_activations_df, current_layer, current_unit, current_percentile, current_task

        current_unit_value = unit_widget.value

        if layer is not current_layer or current_layer_activations_df is None:
            current_layer = layer
            unit_widget.disabled = True
            percentile_widget.disabled = True
            task_widget.disabled = True
            current_layer_activations_df = sg_utils.read_activations_file(activations_dir, layer, force_csv=False)
            unit_widget.options = list(range(len(current_layer_activations_df.columns) - 1))
            unit_widget.disabled = False
            percentile_widget.disabled = False
            task_widget.disabled = False
            current_unit = None

        if current_unit_value is not None and current_unit_value in unit_widget.options:
            unit_widget.value = current_unit_value
        elif unit_widget.value is None:
            unit_widget.value = unit_widget.options[0]

        if unit is not current_unit or current_unit is None:
            current_unit = unit
            if current_unit is not None:
                update_tasks(current_unit, percentile)
            if current_task is not None:
                show_examples(current_task)
        if percentile is not current_percentile or current_percentile is None:
            current_percentile = percentile
            if current_unit is not None:
                update_tasks(current_unit, percentile)
            if current_task is not None:
                show_examples(current_task)
        if task is not current_task or current_task is None:
            current_task = task
            show_examples(current_task)

    prepare_pool()
    interactive_plot = interactive(update_layer_choice, layer=layer_widget, unit=unit_widget, percentile=percentile_widget, task=task_widget)
    output = interactive_plot.children[-1]
    output.layout.height = f'{image_widget.height}px'

    return interactive_plot

In [8]:
def setup_exemplars_ui(model_name):
    activations_dir = os.path.join(ROOT_ACTIVATIONS_DIR, model_name)
    available_layers = list(glob.glob(os.path.join(activations_dir, "*.pkl"))) + list(glob.glob(os.path.join(activations_dir, "*.csv")))
    available_layers = sorted(list(set([os.path.basename(al)[:-4] for al in available_layers])))

    if len(available_layers) == 0:
        raise ValueError(f"Could not find activations for model {model_name}")

    layer_widget = widgets.Dropdown(options=available_layers, description="Layer:")
    unit_widget = widgets.Dropdown(options=[], description="Unit:")
    topk_widget = widgets.IntSlider(min=4, max=100, step=4, value=20, description="#Images:", continuous_update=False)
    image_widget = widgets.Image(value=memoryview(b""), width=800, height=100)

    current_layer_activations_df = None
    current_layer = None
    current_unit = None
    current_topk = None
    current_task = None

    def show_exemplars():
        if current_layer_activations_df is None or current_unit is None or current_topk is None:
            return
        layer_widget.disabled = unit_widget.disabled = topk_widget.disabled = True
        data, (height, width) = get_exemplars_image(*get_exemplar_paths(current_layer_activations_df, current_unit, current_topk))
        image_widget.value = memoryview(data)
        image_widget.height = 8 / width * height * 100
        if interactive_plot:
            interactive_plot.children[-1].layout.height = f'{image_widget.height}px'
        display(image_widget)
        layer_widget.disabled = unit_widget.disabled = topk_widget.disabled = False

    def update_layer_choice(layer, unit, topk):
        nonlocal current_layer_activations_df, current_layer, current_unit, current_topk

        current_unit_value = unit_widget.value

        if layer is not current_layer or current_layer_activations_df is None:
            current_layer = layer
            unit_widget.disabled = topk_widget.disabled = True
            current_layer_activations_df = sg_utils.read_activations_file(activations_dir, layer, force_csv=False)
            unit_widget.options = list(range(len(current_layer_activations_df.columns) - 1))
            unit_widget.disabled = topk_widget.disabled = False
            current_unit = None

        if current_unit_value is not None and current_unit_value in unit_widget.options:
            unit_widget.value = current_unit_value
        elif unit_widget.value is None:
            unit_widget.value = unit_widget.options[0]

        if unit is not current_unit or current_unit is None:
            current_unit = unit
            show_exemplars()
        if topk is not current_topk or current_topk is None:
            current_topk = topk
            show_exemplars()

    prepare_pool()
    interactive_plot = None
    interactive_plot = interactive(update_layer_choice, layer=layer_widget, unit=unit_widget, topk=topk_widget)
    interactive_plot.children[-1].layout.height = f'{image_widget.height}px'

    return interactive_plot

# UI

In [11]:
ui = setup_exemplars_ui(model_name="googlenet")
ui

interactive(children=(Dropdown(description='Layer:', options=('mixed3a_1x1_pre_relu_conv', 'mixed3a_3x3_bottle…

In [13]:
ui = setup_task_ui(model_name="googlenet")
ui

interactive(children=(Dropdown(description='Layer:', options=('mixed3a_1x1_pre_relu_conv', 'mixed3a_3x3_bottle…